# This code implements the training pipeline for various models

In [5]:
import torch
import torch.nn as nn
from torch.optim import Adam
from dataformatter import *
from models import *
import matplotlib.pyplot as plt
import os
import pdb
from collections import defaultdict

In [6]:
# Listing all the hyper-parameters that need to be set here
BATCH_SZ = 512
MODEL_TYPE = 'TRANSFORMER' # OPTIONS ARE : [FC, TRANSFORMER, LSTM]
LR = 5e-4
DATA_PATH = 'hawkeye_trace_belady_graph.csv' # This is the CSV FILE WE ARE TRYING TO ANALYZE
SAVE_FLDR = 'results'
N_EPOCHS = 30
MODEL_DESC='{} MODEL'.format(MODEL_TYPE)

In [7]:
def run_epoch(model, optimizer, data_iterator, mode='train'):
    model.train()
    if mode == 'val':
        model.eval()
    stats = []
    num_egs = 0
    for batch in data_iterator:
        # we get the loss from passing the batch to the model
        # each model will have it's own way of deadling with the data [we can jointly figure this out]
        loss, acc, bsz = model(batch)
        stats.append([loss.item(), acc.item()])
        num_egs += bsz
        if mode == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    stats = np.array(stats)
    avg_loss = np.mean(stats[:, 0])
    avg_acc = (stats[:, 1]).sum() / num_egs
    return avg_loss, avg_acc

In [8]:
def trainer(model, optimizer, dataset, num_epochs=20, desc='Description of model', shuffle=True):
    # Todo [all]
    # Figure out how to split the data into a train-val-test regime
    stats = []
    for epoch_ in range(num_epochs):
        # get a data iterator for this epoch
        data_iter = get_batch_iterator(dataset, BATCH_SZ, shuffle=shuffle)
        epoch_stats = run_epoch(model, optimizer, data_iter, mode='train')
        stats.append(epoch_stats)
        print('Epoch {} : Avg Loss = {} Avg Acc = {}'.format(epoch_, stats[-1][0], stats[-1][1]))
    stats = np.array(stats)
    graph_results(stats, desc)
    return model

def set_wise_trainer(model, optimizer, setwise_dataset, num_epochs=20, desc='Set-Wise Model', shuffle=False):
    model.train()
    stats = defaultdict(list)
    for epoch_ in range(num_epochs):
        # get a data iterator for this epoch
        accs = []
        for set_id, dataset in setwise_dataset.items():
            model.remap_embedders(dataset, set_id)
            data_iter = get_batch_iterator(dataset, BATCH_SZ, shuffle=shuffle)
            this_stats = run_epoch(model, optimizer, data_iter, mode='train')
            stats[set_id].append(this_stats)
            accs.append(this_stats[-1])
        acc_stats = np.min(accs), np.mean(accs), np.median(accs), np.max(accs)
        print('Min Acc {} | Mean Acc : {} | Median Acc {} | Max Acc {} '.format(*acc_stats))
    return model, stats

def evaluate(model, dataset, epoch_id=-1, print_res=True, shuffle=False):
    # get a data iterator for this epoch
    model.eval()
    data_iter = get_batch_iterator(dataset, BATCH_SZ, shuffle=shuffle)
    epoch_stats = run_epoch(model, optimizer, data_iter, mode='test')
    if print_res:
        print('Epoch {} : Avg Loss = {} Avg Acc = {}'.format(epoch_, epoch_stats[0], epoch_stats[1]))
    return epoch_stats

In [9]:
if not os.path.exists(SAVE_FLDR):
    os.makedirs(SAVE_FLDR)

model = get_model(MODEL_TYPE)
optimizer = Adam(model.parameters(), lr=LR)
chosen_columns = model.get_data_columns()
dataset = csv_to_data(DATA_PATH, chosen_columns)
print(model.feat_idx_map)
print(torch.cuda.is_available())

{'Program Counter': 0, 'Set': 1, 'Cache Friendly': 2}
True


In [ ]:
model.prep_for_data(dataset)
average_pred = np.mean(dataset[:, -1])
if torch.cuda.is_available():
    model.cuda()
model.use_cuda = torch.cuda.is_available()
print('This is the average accuracy : ', 1.0 - average_pred, ' From predicting all zeros')
model = trainer(model, optimizer, dataset, num_epochs=N_EPOCHS, desc=MODEL_DESC)

This is the average accuracy :  0.7159665140267546  From predicting all zeros


/projects/tir4/users/ldery/anaconda3/envs/meta4da/lib/python3.6/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([14336])) that is different to the input size (torch.Size([14336, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch 0 : Avg Loss = 0.614604903945645 Avg Acc = 0.7108714702886616
Epoch 1 : Avg Loss = 0.5989321776964132 Avg Acc = 0.7156103718793343
Epoch 2 : Avg Loss = 0.5983402642231543 Avg Acc = 0.7155663520717753
Epoch 3 : Avg Loss = 0.5969935913109085 Avg Acc = 0.7170176512656034
Epoch 4 : Avg Loss = 0.5977325167470765 Avg Acc = 0.7157691818004508
Epoch 5 : Avg Loss = 0.5972964757854499 Avg Acc = 0.716375300689147
Epoch 6 : Avg Loss = 0.597133259460764 Avg Acc = 0.7162151363124133
Epoch 7 : Avg Loss = 0.5969956187368597 Avg Acc = 0.7165943838852289
Epoch 8 : Avg Loss = 0.5972772084393547 Avg Acc = 0.7161003461988558


In [ ]:
setwise_dataset = group_by_set(dataset)
keys = list(setwise_dataset.keys())
num_tr = int(0.9 * len(keys))
train_keys, val_keys = keys[:num_tr], keys[num_tr:]
vals = list([len(x) for k, x in setwise_dataset.items()])
max_key = keys[np.argmax(vals)]
print('Max key {}. Min {}, Mean {}, Median {}, Max {}'.format(max_key, np.min(vals), np.mean(vals), np.median(vals), np.max(vals)))
model.prep_for_data(setwise_dataset[max_key])
print(len(train_keys), len(val_keys))
# Logging-pre-training performance
print('Logging Pre-Training Performance')
all_stats = []
for id_, set_id in enumerate(val_keys):
    if (id_ + 1) % int(len(val_keys) // 5) == 0:
        print(id_, np.array(all_stats).mean(axis=0))
    this_dataset = setwise_dataset[set_id]
    model.remap_embedders(this_dataset, set_id)
    result = evaluate(model, this_dataset, print_res=False)
    average_pred = np.mean(this_dataset[:, -1])
    all_stats.append([*result, 1.0 - average_pred])
av_res = np.mean(all_stats, axis=0)
print('Average Stats Before Training : ', av_res)
train_setwise = {k: setwise_dataset[k] for k in train_keys}
model, stats = set_wise_trainer(model, optimizer, train_setwise, num_epochs=N_EPOCHS)

print('Logging Post-Training Performance')
all_stats = []
for id_, set_id in enumerate(val_keys):
    if (id_ + 1) % int(len(val_keys) // 5) == 0:
        print(id_, np.array(all_stats).mean(axis=0))
    this_dataset = setwise_dataset[set_id]
    model.remap_embedders(this_dataset, set_id)
    result = evaluate(model, this_dataset, print_res=False)
    average_pred = np.mean(this_dataset[:, -1])
    all_stats.append([*result, 1.0 - average_pred])
av_res = np.mean(all_stats, axis=0)
print('Average Stats After Training : ', av_res)

Max key 1807.0. Min 25, Mean 51.50244140625, Median 50.0, Max 115
1843 205
Logging Pre-Training Performance
40 [0.87297585 0.45507812 0.70132639]
81 [0.86061046 0.45904064 0.68750592]
122 [0.85533392 0.46460895 0.68938448]
163 [0.8644205  0.4499297  0.70283787]
204 [0.87130949 0.44881025 0.70726576]
Average Stats Before Training :  [0.87155967 0.4488313  0.70706772]
Min Acc 0.25 | Mean Acc : 0.624512368882914 | Median Acc 0.625 | Max Acc 0.96875 
Min Acc 0.28125 | Mean Acc : 0.6436975470581894 | Median Acc 0.65625 | Max Acc 1.0 
Min Acc 0.25 | Mean Acc : 0.6441128122977651 | Median Acc 0.65625 | Max Acc 0.96875 
Min Acc 0.25 | Mean Acc : 0.6455715436343803 | Median Acc 0.65625 | Max Acc 1.0 
Min Acc 0.25 | Mean Acc : 0.6436549225179562 | Median Acc 0.65625 | Max Acc 1.0 
Min Acc 0.25 | Mean Acc : 0.6449629183340161 | Median Acc 0.65625 | Max Acc 0.96875 
Min Acc 0.25 | Mean Acc : 0.6446413525550112 | Median Acc 0.65625 | Max Acc 0.96875 
Min Acc 0.25 | Mean Acc : 0.6443906613941192 | M